# Vector Space Model 
### Cristian Curaba

In [1]:
# Importing dependancy libraries
import os
import pandas as pd
import numpy as np
import re
import math as m
from collections import Counter
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
# Defining global variables for directories
in_path_files=os.getcwd()+"/cranfieldDocs"
out_path_files=os.getcwd()+"/preprocessed_cranfieldDocs"
path=os.getcwd()

if not os.path.isdir(out_path_files):
    os.mkdir(out_path_files)


**Preprocessing Steps:**

- Transform the input text to lowercase.
- Remove punctuation, numbers.
- Exclude words with one or two characters in length.
- Perform stemming.
- Eliminate stopwords.


In [3]:
# Initiallizing Porter Stemmer object
st = PorterStemmer()

#Intializing regular expression object to remove words with one or two characters length
shortword = re.compile(r'\W*\b\w{1,2}\b')

# Intializing stopwords list
stop_list = stopwords.words('english')


def preprocess_text(input_text):
    """
    Preprocesses the input text. Converts to lower case,
    removes punctuation and numbers, splits on whitespaces, 
    removes stopwords, performs stemming & removes words with one or 
    two characters length.

    Arguments:
        input_text {string} -- text to be tokenized

    Returns:
        string -- string of tokens generated
    """

    # Converting to lower case
    text_lower = input_text.lower()

    # Removing anythig that is not a word character or a whitespace
    text_no_punct = re.sub(r'[^\w\s]', '', text_lower)

    # Removing numbers
    text_no_numbers = re.sub(r'[0-9]', '', text_no_punct)

    # Removing tokens with one or two characters length
    text_no_short_words = shortword.sub('', text_no_numbers)

    # Splitting on whitespaces to generate tokens
    tokens = text_no_numbers.split()

    # Removing stop words from the tokens
    clean_tokens = [word for word in tokens if word not in stop_list]

    # Stemming the tokens
    stem_tokens = [st.stem(word) for word in clean_tokens]

    # Checking for stopwords again
    clean_stem_tokens = [word for word in stem_tokens if word not in stop_list]

    # Converting list of tokens to string
    clean_stem_tokens_final = ' '.join(map(str,  clean_stem_tokens))
    

    return clean_stem_tokens_final


In [4]:
#Utility function to extract tokens from a file
def extractTokens(beautifulSoup, tag):
    """Extract tokens of the text between a specific SGML <tag>.

    Arguments:
        beautifulSoup {bs4.BeautifulSoup} -- soup bs object formed using text of a file
        tag {string} -- target SGML <tag>

    Returns:
        string -- string of tokens extracted from text between the target SGML <tag>
    """

    # find the tag and get its text content
    tag_content = beautifulSoup.find(tag).text if beautifulSoup.find(tag) else ''

    return tag_content


Performed pre-processing phase to corpus and queries.

Here I joined title and text of docs toghether, an improvement is possible: giving more relevance for title occurences.

In [5]:
filenames = os.listdir(in_path_files)

for fname in filenames:

    # generate filenames
    infilepath = in_path_files + '/' + fname
    outfilepath = out_path_files + '/' + fname

    with open(infilepath) as infile:
        with open(outfilepath, 'w') as outfile:

            # read all text in a file
            fileData = infile.read()

            # creating BeautifulSoup object to extract text between SGML tags
            soup = BeautifulSoup(fileData)

            # extract tokens for <title>
            title = extractTokens(soup, 'title')

            # extract tokens for <text>
            text = extractTokens(soup, 'text')

            # preprocess tokens for <title>
            title = preprocess_text(title)

            # preprocess tokens for <text>
            text = preprocess_text(text)
            
            # write tokens for <title> into new file
            outfile.write(title)
            outfile.write(" ")

            # write tokens for <text> into new file
            outfile.write(text)
        outfile.close()
    infile.close()

print("Preprocessing files done!")

#Preprocessing query file
query_file=os.getcwd()+"/queries.txt"
query_file_out=os.getcwd()+"/preprocessed_queries.txt"

# Preprocessing the queries.txt file
q = open(query_file, 'r')

# opening new file to write preprocessed tokens into
new_q = open(query_file_out, 'w')

# read each line of file seperately
text = q.readlines()
for line in text:
    
    # if condition to avoid newline character in the end of file
    if(line != text[-1]):
        query_tokens = preprocess_text(line)
        new_q.write(query_tokens + '\n')
    else:
        query_tokens = preprocess_text(line)
        new_q.write(query_tokens)

q.close()
new_q.close()

print("Preprocessing queries done!")

Preprocessing files done!
Preprocessing queries done!


In [6]:
# Generate a single list of all preprocessed docs
all_docs = []

for fname in filenames:
    outfilepath = out_path_files + '/' + fname
    with open(outfilepath) as file:
        fileData = file.read()
        all_docs.append(fileData)
    file.close()

### Creating tf-idf dictionary.

Build $\texttt{DF}$ dictionary with pairs $\texttt{(token, number of occurences)}$.

In [7]:
from collections import defaultdict
#Calculating document frequency
# Create a defaultdict with sets as default values
DF = defaultdict(set)

# Populate the dictionary with document occurrences for each token
for doc_index, doc in enumerate(all_docs):
    tokens = set(doc.split())
    for token in tokens:
        DF[token].add(doc_index)

# Convert the sets to the number of occurrences
DF = {token: len(doc_set) for token, doc_set in DF.items()}


Here I calculate the $\texttt{tf\_idf}= \texttt{tf}\times \texttt{idf}$ dictionary with all pairs $(\texttt{token, tf\_idf})$ where $$\texttt{tf}=\frac{\texttt{freq}_{t,d}}{\texttt{len}(d)}$$
 with $t$ token, $d$ document and $\texttt{freq}_{t,d}$ is the number of occurence of token $t$ in the document $d$;
$$
idf_t= \log \frac{N}{df_t}, 
$$ 
with $t$ token, $N$ nummber of docs and $df_t$ the document frequency of the token $t$.

In [8]:
#Calculating tf-idf
# NB: tf-idf is a term-document set of values given by  the product of tf which is the term frequency (tf_{t,d}= f_{t,d}/len(d))
# and idf which is the inverse document frequency, i.e. the logarithm of the number of documents divided by the number of documents containing the term.


# vocabulary of all the terms in the corpus
vocab = [term for term in DF]
# creating dictionary to store tf-idf values for each term in the vocabulary
tf_idf = {}

#doc is the index of the document in the corpus
doc = 0

for i in range(len(all_docs)):
    # tokenizing each document
    tokens = all_docs[i].split()
    
    # counter object to efficiently count number of occurence of a term in a particular document
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        # counting occurence of term in document using counter object
        tf = counter[token]/words_count
        
        # retrieving df values from DF dictionary
        df = DF[token] if token in vocab else 0
        
        # adding 1 to numerator & denominator to avoid divide by 0 error
        idf = np.log((len(all_docs)+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf
    doc += 1
    
#printing some tf-idf values
print(f'tf_idf value of \'approach\' in doc number 15: ', tf_idf[14, 'approach'])
print(f'tf_idf value of \'experi\' in doc number 1: ', tf_idf[0, 'experiment'])
print(f'tf_idf value of \'stress\' in doc number 1400: ', tf_idf[1399, 'stress'])

tf_idf value of 'approach' in doc number 15:  0.047682688711314716
tf_idf value of 'experi' in doc number 1:  0.00861754939911508
tf_idf value of 'stress' in doc number 1400:  0.08963980961566191


## Calculate cosine similarity

In [9]:
# Building the tf-idf document-term matrix with numpy
# initializing empty vector of vocabulary size
# TF_IDF[0] is empty
TF_IDF = np.zeros((len(all_docs), len(vocab)))

# creating vector of tf-idf values
for i in tf_idf:
    ind = vocab.index(i[1])
    TF_IDF[i[0]][ind] = tf_idf[i]


In the next code snippet, we will calculate the cosine similarity between two vectors using the $\texttt{cosine\_similarity} $ function. This function takes two numpy arrays as input and returns the cosine similarity between them. We will also define the $\texttt{ranking}$ function, which will determine a ranked list of top k documents based on their cosine similarity with a given query. Finally, we will use the $\texttt{generate\_ranked\_document\_list}$ function to generate a ranked list of documents in descending order of their cosine similarity with the queries.


In [10]:
#Utility function to generate vector representation of tokens where v(d)\in R^|V|, [v(d)]_i = tf_{t_i,d} * idf_{t_i}
def generate_vector(tokens):
    """
    Create a vector based on the vocabulary for the given tokens.
    
    Args:
        tokens (list): List of tokens to be converted.
    
    Returns:
        numpy.ndarray: Vector representation of tokens.
    """
    vector = np.zeros(len(vocab))
    token_counts = Counter(tokens)
    total_words = len(tokens)

    for token in np.unique(tokens):
        term_frequency = token_counts[token] / total_words
        document_frequency = DF[token] if token in vocab else 0
        inverse_document_frequency = m.log((len(all_docs) + 1) / (document_frequency + 1))

        try:
            index = vocab.index(token)
            vector[index] = term_frequency * inverse_document_frequency
        except ValueError:
            pass

    return vector

#Utility function to calculate cosine similarity between 2 vectors
def cosine_similarity(x, y):
    """Calculate cosine similarity between 2 vectors (same dimension).
    Arguments:
        x {numpy.ndarray} -- vector 1
        y {numpy.ndarray} -- vector 2
    
    Returns:
        numpy.float64 -- cosine similarity between vector 1 & vector 2
    """
    if not (np.issubdtype(x.dtype, np.number) and np.issubdtype(y.dtype, np.number)):
        raise ValueError("Input vectors must have numeric data types.")
    if x.shape != y.shape:
        raise ValueError("Input vectors must have the same dimensions.")
    
    if np.all(x == 0) or np.all(y == 0):
        return 0
    return np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

# Utility function to rank documents based on cosine similarity with query
def ranking(k, query):
    """Determins a ranked list of top k documents in descending order of their
    cosine similarity with the query
    
    Arguments:
        k {integer} -- top k documents to retrieve from (if k=0, retrieve all documents)
        query {string} -- query whose cosine similarity is to be computed with the corpus
    
    Returns:
        numpy.ndarray -- list of top k cosine similarities between query and corpus of documents
    """
    if(k<0):
        print("Please enter a value >=0")
        return
    
    tokens = query.split()
    #print(tokens)
    d_cosines = []
    
    # vectorize the input query tokens
    query_vector = generate_vector(tokens)
    
    for d in TF_IDF:
        d_cosines.append(cosine_similarity(query_vector, d))
        
    if k == 0:
        # k=0 to retrieve all documents in descending order
        out = np.array(d_cosines).argsort()[::-1]
        
    else:
        # to retrieve the top k documents in descending order    
        out = np.array(d_cosines).argsort()[-k:][::-1]
    
    return out
    

In the next snippet there's the function to generate the ranked retrived list (of k documents) for each query.

In [11]:
# queries is the list of all queries from preprocessed queries file
queries = open(query_file_out, 'r').readlines()
def generate_ranked_document_list(top_k=0):
    """
    Generate a ranked list of top k documents in descending order of their cosine similarity
    calculated against the queries. If k=0 is given, return the list of all documents
    in descending order.

    Args:
        top_k (int): Number of top documents to be retrieved.

    Returns:
        list: List of documents in descending order of their cosine similarity.
              Each element is a tuple (query_id, document_id).
    """
    cosine_similarities = []

    for query_id in range(len(queries)):
        similarity_score = ranking(top_k, queries[query_id])
        cosine_similarities.append(similarity_score)

    return cosine_similarities

# generating ranked list of top 10 documents for each query
print(generate_ranked_document_list(10))

[array([ 917,  172,  566,  449,  436,  754,  365,  739,  455, 1243]), array([ 949,  673,  570,  228, 1305,  866,  714,  344,  171,  906]), array([ 949,  673,  228,  866,  570, 1305,  714, 1363,  139,  858]), array([ 795,  914,  281, 1255,  552,  649,  385,  441,  888,  195]), array([303, 697, 680, 996, 133,  93, 166, 775, 524, 661]), array([1186,  315,  436, 1072,  721,  707,  133,  856,  697,   65]), array([1018,  965,  299,  541,  507,  567,  580, 1087,  513,  726]), array([726, 541, 574, 965, 577, 315, 697, 766, 720, 127]), array([1153,  803, 1108,  754, 1303, 1085,  780,  725, 1013,  326]), array([ 777, 1324, 1313,  297,  411,  105, 1210,  621,  750,  972])]


## Vector Model Evaluation: precision and recall.

In [12]:
#Get the relevance of each document for each query
# NB: queries and docs numbered from 1.
column_names = ['Query_ID', 'Relevance']
relevance_df = pd.read_csv(f"{path}/relevance.txt", delim_whitespace=True, names=column_names, header=None)
relevance_df['Relevance']=relevance_df['Relevance'].astype(int)
relevance_df=relevance_df - 1 # to make the indexing of queries and docs start from 0
print(relevance_df.head())


query_relevance_lists = []
for query_id in range(len(queries)):
    relevant_documents = relevance_df[relevance_df['Query_ID'] == query_id]['Relevance'].to_list()
    query_relevance_lists.append(relevant_documents)
print(query_relevance_lists)

   Query_ID  Relevance
0         0        155
1         1        665
2         1        666
3         1       1257
4         1       1393
[[155], [665, 666, 1257, 1393, 667, 669, 1203, 1390, 1394, 1299, 36, 558, 629, 1106, 1212], [23, 100, 665, 666, 92, 1257, 1392, 558, 629, 661, 1103, 1106, 1203, 1212, 1299], [1390, 665, 666, 1257, 1077, 1079, 1080, 1393, 1394, 1213, 1197, 1203, 1299, 558, 629, 661, 1106, 1212], [1382, 1384, 154, 240, 1381, 1369, 1385, 110, 1383, 149, 291, 457, 478, 976, 375, 458, 1364, 61, 1365], [154, 1382, 1384, 1381, 61, 291, 240, 1369, 1383, 457, 458, 460, 1385, 1364, 1365, 110, 149, 478], [399, 418, 1386, 411, 1391, 1397, 1396, 1399, 1398], [399, 1386, 1391, 1397], [655, 1312, 1316, 1315, 1317, 1318, 1156, 1273], [1378, 1304, 1303, 39, 292, 1308, 160, 420, 1376, 1377, 1380, 224, 1379, 447, 448, 1123, 1279, 432, 922, 923, 1061, 1073, 1074, 1212]]


In [13]:
# Calculating recall= number of relevant documents retrieved/total number of relevant documents

# Get the relevance of each document for each query
def calculate_recall_at_k(k=50):
    """
    Calculate recall values for each query given the number of top documents to be retrieved (k).

    Args:
        k (int): Number of top documents to be retrieved.

    Returns:
        list: List of recall values for each query.
    """
    recall_values = []

    for query_id in range(len(queries)):
        relevant_documents = query_relevance_lists[query_id]
        top_k_documents = generate_ranked_document_list(k)[query_id]

        # Number of relevant documents retrieved
        intersect_count = len([value for value in top_k_documents if value in relevant_documents])

        # Total number of relevant documents
        total_relevant_documents = len(relevant_documents)

        # Calculate recall for the current query
        recall = intersect_count / total_relevant_documents
        recall_values.append(recall)

    return recall_values

recall_results = calculate_recall_at_k(50)
print(recall_results)


[0.0, 0.0, 0.0, 0.0, 0.10526315789473684, 0.0, 0.1111111111111111, 0.0, 0.0, 0.041666666666666664]


In [14]:
#Calculating precision= number of relevant documents retrieved/total number of documents retrieved
def calculate_precision_at_k(k=50):
    """To generate list of precision values for each query for given value of k
    
    Arguments:
        k {[type]} -- number of top documents to be retrieved
    
    Returns:
        list -- list of precision values for each query
    """
    precision_values = []
    for query_id in range(len(queries)):
        relevant_documents = query_relevance_lists[query_id]
        top_k_documents = generate_ranked_document_list(k)[query_id]

        # Number of relevant documents retrieved
        intersect_count = len([value for value in top_k_documents if value in relevant_documents])

        # Calculate recall for the current query
        precision = intersect_count / k
        precision_values.append(precision)

    return precision_values
calculate_precision_at_k(50)

[0.0, 0.0, 0.0, 0.0, 0.04, 0.0, 0.02, 0.0, 0.0, 0.02]

In [15]:
# Writing precision and recall values to file
list_of_k = [5, 10, 20, 50, 100]
# If file already exists, delete it
if not os.path.exists('accuracy_without_feedbacks'):
    with open('accuracy_without_feedbacks', 'w') as file:
        for k in list_of_k:
            p = calculate_precision_at_k(k)
            r = calculate_recall_at_k(k)
            
            file.write(f"Top {k} documents in the rank list\n")
            
            for i in range(len(queries)):
                approximated_recall = round(r[i], 2)
                file.write("Query: {0} \t Pr: {1} \t Re: {2}\n".format(i+1, p[i], approximated_recall))
            
            avg_precision = round(np.mean(p), 2)
            avg_recall = round(np.mean(r), 2)
            
            file.write("Avg Precision: {0}\n".format(avg_precision))
            file.write("Avg Recall: {0}\n\n".format(avg_recall))


## Relevance Feedback

In [16]:
#Generating ranked list of top k documents for each query after relevance feedback

# query_relevance_lists[query_id] is the list of relevant documents for the query_id

In [17]:
#Rocchio Algorithm
# Intializing alpha, beta, gamma values
alpha = 1
beta = 0.75
gamma = 0.15

# Relevant documents for each query
relevant_docs= query_relevance_lists #here using the relevance list given by the dataset

# Calculate centroid of relevant/irrelevant documents
def calculate_centroid(docs):
    """
    Calculate centroid of relevant/irrelevant documents.

    Args:
        relevant_documents (list): List of relevant documents.

    Returns:
        numpy.ndarray: Centroid vector of relevant documents.
    """
    centroid = np.zeros(len(vocab))

    for document_id in docs:
        centroid += TF_IDF[document_id]

    return centroid / len(docs)


def calculate_new_query_vector(query_id, alpha=1, beta=0.75, gamma=0.15):
    """
    Calculate new query vector based on Rocchio algorithm.

    Args:
        query_id (int): Query ID.
        centroid (numpy.ndarray): Centroid vector of relevant documents.
        alpha (float): Weight for the original query vector.
        beta (float): Weight for the centroid of relevant documents.
        gamma (float): Weight for the centroid of non-relevant documents.

    Returns:
        numpy.ndarray: New query vector.
    """
    original_query_vector = generate_vector(queries[query_id].split())
    # Get the list of relevant and non-relevant document IDs for the current query
    relevant_document_ids = relevant_docs[query_id]
    non_relevant_document_ids = [doc_id for doc_id in range(len(all_docs)) if doc_id not in relevant_document_ids]
    print(non_relevant_document_ids)
    # Calculate Rocchio algorithm components
    relevant_centroid = calculate_centroid(relevant_document_ids)
    non_relevant_centroid = calculate_centroid(non_relevant_document_ids)

    # Calculate new query vector using Rocchio algorithm
    new_query_vector = alpha * original_query_vector + beta * relevant_centroid - gamma * non_relevant_centroid

    return new_query_vector

def generate_ranked_document_list_with_feedback(query_id, top_k=0):
    """
    Generate a ranked list of top k documents in descending order of their cosine similarity
    calculated against the queries. If k=0 is given, return the list of all documents
    in descending order.

    Args:
        query_id (int): Query ID.
        top_k (int): Number of top documents to be retrieved.

    Returns:
        list: List of documents in descending order of their cosine similarity.
              Each element is a tuple (query_id, document_id).
    """
    cosine_similarities = []

    for query_id in range(len(queries)):
        new_query_vector = calculate_new_query_vector(query_id)

        # Calculate cosine similarity between new query vector and all document vectors
        for document_vector in TF_IDF:
            similarity_score = cosine_similarity(new_query_vector, document_vector)
            cosine_similarities.append(similarity_score)
        cosine_similarities.append(similarity_score)

    return cosine_similarities

In [18]:
# Calculating recall= number of relevant documents retrieved/total number of relevant documents

# Get the relevance of each document for each query
def calculate_recall_at_k_with_feedback(k=50):
    """
    Calculate recall values for each query given the number of top documents to be retrieved (k).

    Args:
        k (int): Number of top documents to be retrieved.

    Returns:
        list: List of recall values for each query.
    """
    recall_values = []

    for query_id in range(len(queries)):
        relevant_documents = query_relevance_lists[query_id]
        top_k_documents = generate_ranked_document_list_with_feedback(query_id, k)

        # Number of relevant documents retrieved
        intersect_count = len([value for value in top_k_documents if value in relevant_documents])

        # Total number of relevant documents
        total_relevant_documents = len(relevant_documents)

        # Calculate recall for the current query
        recall = intersect_count / total_relevant_documents
        recall_values.append(recall)

    return recall_values

recall_results = calculate_recall_at_k_with_feedback(50)
print(recall_results)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,

In [19]:
#Calculating precision= number of relevant documents retrieved/total number of documents retrieved
def calculate_precision_at_k_with_feedback(k=50):
    """To generate list of precision values for each query for given value of k
    
    Arguments:
        k {[type]} -- number of top documents to be retrieved
    
    Returns:
        list -- list of precision values for each query
    """
    precision_values = []
    for query_id in range(len(queries)):
        relevant_documents = query_relevance_lists[query_id]
        top_k_documents = generate_ranked_document_list_with_feedback(k)[query_id]

        # Number of relevant documents retrieved
        intersect_count = len([value for value in top_k_documents if value in relevant_documents])

        # Calculate recall for the current query
        precision = intersect_count / k
        precision_values.append(precision)

    return precision_values
calculate_precision_at_k(50)

[0.0, 0.0, 0.0, 0.0, 0.04, 0.0, 0.02, 0.0, 0.0, 0.02]